In [1]:
from web3 import Web3
import json
from eth_account import Account


web3 = Web3(Web3.HTTPProvider(""))




private_key = ""


ACC_ADDRESS = Account.from_key(private_key).address

SETUP_ADDR = ""
setup_contract = web3.eth.contract(
        address=Web3.toChecksumAddress(SETUP_ADDR), abi=json.load(open("setup.json", "r"))
    )

In [2]:
DISTRIBUTOR_ADDR = setup_contract.functions.merkleDistributor().call()
print(DISTRIBUTOR_ADDR)
distributor_contract = web3.eth.contract(address=DISTRIBUTOR_ADDR, abi=json.load(open("distributor.json", "r")))

0xfaF64f7718b3CdcFd2C90e6811608656b9B25737


In [3]:
tree = json.load(open("../tree.json", "r"))


In [4]:
def send_claim_tx(index, account, amount, merkle_proof):
    nonce = web3.eth.get_transaction_count(ACC_ADDRESS)
    print("Sending index:", index,"Nonce:", nonce)

    transaction = distributor_contract.functions.claim(index, account, amount, merkle_proof).buildTransaction({
        "value": 0,
        'nonce': nonce,
        'maxFeePerGas': 0,
        'maxPriorityFeePerGas': 0,
        'from': ACC_ADDRESS
    }
    )

    signed_transaction = web3.eth.account.sign_transaction(
        transaction, private_key=private_key)

    res = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)

    receipt = web3.eth.wait_for_transaction_receipt(res)


In [5]:
special_amount = 0
for acc, info in tree["claims"].items():
    if info["index"] == 8:
        send_claim_tx(info["index"], acc, int(info["amount"], 16), info["proof"])

Sending index: 8 Nonce: 0


In [7]:
send_claim_tx(int("0xd43194becc149ad7bf6db88a0ae8a6622e369b3367ba2cc97ba1ea28c407c442", 16),
              web3.toChecksumAddress("0xd48451c19959e2d9bd4e620fbe88aa5f6f7ea72a"), int("0x00000f40f0c122ae08d2207b", 16), [
    "0x8920c10a5317ecff2d0de2150d5d18f01cb53a377f4c29a9656785a22a680d1d",
    "0xc999b0a9763c737361256ccc81801b6f759e725e115e4a10aa07e63d27033fde",
    "0x842f0da95edb7b8dca299f71c33d4e4ecbb37c2301220f6e17eef76c5f386813",
    "0x0e3089bffdef8d325761bd4711d7c59b18553f14d84116aecb9098bba3c0a20c",
    "0x5271d2d8f9a3cc8d6fd02bfb11720e1c518a3bb08e7110d6bf7558764a8da1c5"
])


Sending index: 95977926008167990775258181520762344592149243674153847852637091833889008632898 Nonce: 1


In [8]:
print(setup_contract.functions.isSolved().call())

True
